<a href="https://colab.research.google.com/github/iptkachev/yda_crowdsourcing/blob/main/hw2/toloka_pool_prepare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q toloka-kit==0.1.17 # To interact with Toloka API
!pip install -q ipyplot # To plot images inside Jupyter Notebooks cells
!pip install -q crowd-kit==0.0.7
!pip install -q yadisk
import os
import datetime
import time
import logging
import sys

import pandas as pd   # To perform data manipulation
import ipyplot
from IPython.display import clear_output

from typing import List
from toloka.streaming.event import AssignmentEvent

import toloka.client as toloka
import toloka.client.project.template_builder as tb

from crowdkit.aggregation import MajorityVote

logging.basicConfig(
    format='[%(levelname)s] %(name)s: %(message)s',
    level=logging.INFO,
    stream=sys.stdout,
)

In [ ]:
! curl https://tlk.s3.yandex.net/course/shad_cv_project_22/dataset_students.zip --output dataset_students.zip
! unzip -q dataset_students.zip
! ls dataset_students/classification/train_unlabelled | wc -l

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  400M  100  400M    0     0  15.2M      0  0:00:26  0:00:26 --:--:-- 17.1M
8500


In [ ]:
import yadisk
y = yadisk.YaDisk(token=input())
clear_output()
print(y.check_token())

True


In [ ]:
ROOT = "dataset_students/classification/train_unlabelled"

for image in os.listdir(ROOT):
  y.upload(os.path.join(ROOT, image), f"disk:/Приложения/Toloka/project1_cv_copy/{image}")

In [ ]:
toloka_client = toloka.TolokaClient(input("Enter your token:"), 'PRODUCTION') # Or switch to 'SANDBOX'
print(toloka_client.get_requester())
clear_output()

In [ ]:
pool = toloka_client.get_pool(35892859)

In [ ]:
uploaded_images = list(y.listdir("disk:/Приложения/Toloka/project1_cv_copy", timeout=100))
uploaded_images_names = sorted(list(map(lambda x: x['name'], uploaded_images)))

In [ ]:
price = 0.02
limit_budget = 20
len(uploaded_images_names[:int(limit_budget / 0.01)])

2000

In [ ]:
price = 0.02
limit_budget = 20
learn_control_tasks = 40

tasks = [
    toloka.Task(input_values={'image': f'https://platform.toloka.ai/api/proxy/project1-cv-copy/{image}'}, pool_id=pool.id)
    for image in uploaded_images_names[:int(limit_budget / price) + learn_control_tasks]
]

In [ ]:
toloka_client.create_tasks(tasks, allow_defaults=True)
clear_output()